<a href="https://colab.research.google.com/github/bhstoller/multi-agent-customer-service/blob/main/agent_to_agent_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Agent-to-Agent Orchestration**

This notebook contains all of the agent configurations, instructions, agent cards, and implementation logic for the Router agent for the three specialized agents in the system:
- Customer Data Agent: Handles all customer data requests via the MCP Server Tools
- Support Agent: Handles all customer support and escalation requests
- Router (Orchestrator): Facilitates the communication between customer data and support agents (runs as client, not server) via a custom class

This notebook mirrors the code in `src/config.py` (Configuration), `src/agents.py` (Agent definitions and instructions), and `src/router.py` (A2A client and router orchestrator).

## **Configuration (from `src/config.py`)**

In [1]:
!git clone https://github.com/bhstoller/multi-agent-customer-service.git

Cloning into 'multi-agent-customer-service'...
remote: Enumerating objects: 236, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 236 (delta 44), reused 49 (delta 31), pack-reused 165 (from 1)
Receiving objects: 100% (236/236), 129.48 KiB | 1.58 MiB/s, done.
Resolving deltas: 100% (117/117), done.


In [2]:
!pip install -q -r multi-agent-customer-service/requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 10.2 MB/s eta 0:00:00


In [3]:
import os
from pathlib import Path
from google.colab import userdata
import logging

# Set up Google Cloud configurations
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'FALSE'
os.environ['GOOGLE_CLOUD_PROJECT'] = 'app-ai-a5'
os.environ['GOOGLE_CLOUD_LOCATION'] = 'us-central1'
os.environ["GOOGLE_API_KEY"] = userdata.get("a5-key")
LLM_MODEL = os.getenv('LLM_MODEL', 'gemini-2.0-flash')

# Set up NGrok configurations
NGROK_AUTHTOKEN = userdata.get('NGROK_AUTHTOKEN')
USE_NGROK = bool(NGROK_AUTHTOKEN)
MCP_SERVER_URL = userdata.get('MCP_SERVER_URL')

# Set up database configurations
DB_PATH = "/content/support.db"

# Set notebook configurations
LLM_MODEL = "gemini-2.0-flash"
LOG_LEVEL = "CRITICAL"
DEBUG = False

ROUTER_AGENT_NAME = 'router_agent'
SUPPORT_AGENT_NAME = 'support_agent'
CUSTOMER_DATA_AGENT_NAME = 'customer_data_agent'

# Set logging configurations
logging.basicConfig(level=logging.ERROR, force=True)
logging.getLogger('asyncio').setLevel(logging.CRITICAL)
logging.getLogger('aiohttp').setLevel(logging.CRITICAL)

print('Configurations loaded!')

Configurations loaded!


## **Import Dependencies**

In [4]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver

class PatchedClientModule:
    def __init__(self, real_module) -> None:
        for attr in dir(real_module):
            if not attr.startswith('_'):
                setattr(self, attr, getattr(real_module, attr))
        self.A2ACardResolver = A2ACardResolver


patched_module = PatchedClientModule(real_client_module)
sys.modules['a2a.client.client'] = patched_module  # type: ignore

In [5]:
import sys
from a2a.client import client as real_client_module
from a2a.client.card_resolver import A2ACardResolver
import asyncio
import logging
import os
import sys
import threading
import time
from typing import Any
import httpx
import nest_asyncio
import uvicorn
from a2a.client import ClientConfig, ClientFactory, create_text_message_object
from a2a.server.apps import A2AStarletteApplication
from a2a.server.request_handlers import DefaultRequestHandler
from a2a.server.tasks import InMemoryTaskStore
from a2a.types import (
    AgentCapabilities,
    AgentCard,
    AgentSkill,
    TransportProtocol,
)
from a2a.utils.constants import AGENT_CARD_WELL_KNOWN_PATH
from dotenv import load_dotenv
from google.adk.a2a.executor.a2a_agent_executor import (
    A2aAgentExecutor,
    A2aAgentExecutorConfig,
)
from google.adk.agents import Agent, SequentialAgent
from google.adk.agents.remote_a2a_agent import RemoteA2aAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import MCPToolset
from google.adk.tools.mcp_tool.mcp_toolset import StreamableHTTPConnectionParams
import re
import google.generativeai as genai
import os
import json
import asyncio
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.12/dist-packages/google/cloud/aiplatform/models.py:52: FutureWarning: Support for google-cloud-storage < 3.0.0 will be removed in a future version of google-cloud-aiplatform. Please upgrade to google-cloud-storage >= 3.0.0.
  from google.cloud.aiplatform.utils import gcs_utils


## **Agent Definitions (from `src/agents.py`)**

### Customer Data Agent

In [6]:
CUSTOMER_DATA_AGENT_URL = 'http://127.0.0.1:10020'

CUSTOMER_DATA_AGENT_INSTRUCTION = '''You are the Customer Data Agent.

IMPORTANT LOGGING: Begin EVERY response with:
[CUSTOMER_DATA_AGENT]: <brief description of what you're doing>

Your role is to interface with the MCP server and perform EXACT data operations.

REQUIRED RULES:
- ALWAYS announce your action first: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID X
- ALWAYS call an MCP tool for any operation involving customer data.
- NEVER invent fields, values, or IDs.
- ALWAYS return valid JSON after your announcement.
- If a customer does not exist, return: {\"error\": \"Customer ID not found\", \"customer_id\": <id>}
- For successful lookups, return: {\"customer\": { ...mcp result... }}

AVAILABLE OPERATIONS:
- get_customer(customer_id)
- list_customers(status, limit)
- update_customer(customer_id, data)
- create_ticket(customer_id, issue, priority)
- get_customer_history(customer_id)

ADDITIONAL RULES:
- When updating a customer, preserve any fields that are not being modified.
- When creating a ticket, always include created_at returned by MCP.
- When listing customers, return an array of customer objects.

ALWAYS start with [CUSTOMER_DATA_AGENT]: then provide JSON.
'''

customer_data_agent = Agent(
    model=LLM_MODEL,
    name='customer_data_agent',
    tools=[MCPToolset(connection_params=StreamableHTTPConnectionParams(url=MCP_SERVER_URL))],
    instruction=CUSTOMER_DATA_AGENT_INSTRUCTION,
)

customer_data_agent_card = AgentCard(
    name='Customer Data Agent',
    url=CUSTOMER_DATA_AGENT_URL,
    description='Fetches and updates customer data using MCP tools',
    version='1.0',
    capabilities=AgentCapabilities(streaming=False),
    default_input_modes=['text/plain'],
    default_output_modes=['application/json'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[AgentSkill(id='customer_data_access', name='Customer Data Access', description='Retrieve and update customer records', tags=['customer', 'data', 'database', 'lookup'], examples=['Get customer 42', 'Update customer 5 email', 'Show all customers'])],
)

print('Customer Data Agent created')

Customer Data Agent created


### Support Agent

In [7]:
SUPPORT_AGENT_URL = 'http://127.0.0.1:10021'

SUPPORT_AGENT_INSTRUCTION = '''You are the Support Agent.

IMPORTANT LOGGING: Begin EVERY response with:
[SUPPORT_AGENT]: <brief description of what you're doing>

CRITICAL RULE: If the user asks for customer INFORMATION/DATA (get, retrieve, show, lookup),
you MUST respond with:
[SUPPORT_AGENT]: This is a data retrieval request, not a support question.
PASS_TO_CUSTOMER_DATA_AGENT

Only handle SUPPORT questions like:
- \"How do I reset my password?\"
- \"I need help with my account\"
- \"I'm having trouble logging in\"

If the query is asking for customer data/information/records, say:
[SUPPORT_AGENT]: PASS_TO_CUSTOMER_DATA_AGENT

If customer-specific data is required for a SUPPORT question, respond with:
[SUPPORT_AGENT]: Requesting customer data from router
{
  \"needs_customer_data\": true,
  \"reason\": \"<why>\",
  \"requested_fields\": [\"email\", \"tickets\", \"status\", ...]
}

When the Router provides customer data for a support question:
- Announce: [SUPPORT_AGENT]: Processing support request with customer context
- Use only those fields
- Provide a natural-language support answer

Your job: troubleshooting, escalation, and answering support questions ONLY.
For data retrieval queries, defer to customer data agent.

ALWAYS start responses with [SUPPORT_AGENT]:
'''

support_agent = Agent(
    model=LLM_MODEL,
    name='support_agent',
    instruction=SUPPORT_AGENT_INSTRUCTION,
)

support_agent_card = AgentCard(
    name='Support Agent',
    url=SUPPORT_AGENT_URL,
    description='Handles general support questions and escalates to customer data agent when needed',
    version='1.0',
    capabilities=AgentCapabilities(streaming=True),
    default_input_modes=['text/plain'],
    default_output_modes=['text/plain'],
    preferred_transport=TransportProtocol.jsonrpc,
    skills=[AgentSkill(id='customer_support', name='Customer Support', description='Provides general support responses', tags=['support', 'help', 'troubleshooting'], examples=['How do I reset my password?', 'What is your refund policy?', 'I need help with my account.'])],
)

print('Support Agent created')

Support Agent created


## **Router and A2A Client (`from src/router.py`)**

### A2A Simple Client

In [8]:
class A2ASimpleClient:
    """A2A Simple to call A2A servers."""

    def __init__(self, default_timeout: float = 240.0):
        self._agent_info_cache: dict[
            str, dict[str, Any] | None
        ] = {}  # Cache for agent metadata
        self.default_timeout = default_timeout

    async def create_task(self, agent_url: str, message: str) -> str:
        """Send a message following the official A2A SDK pattern."""
        # Configure httpx client with timeout
        timeout_config = httpx.Timeout(
            timeout=self.default_timeout,
            connect=10.0,
            read=self.default_timeout,
            write=10.0,
            pool=5.0,
        )

        async with httpx.AsyncClient(timeout=timeout_config) as httpx_client:
            # Check if we have cached agent card data
            if (
                agent_url in self._agent_info_cache
                and self._agent_info_cache[agent_url] is not None
            ):
                agent_card_data = self._agent_info_cache[agent_url]
            else:
                # Fetch the agent card
                agent_card_response = await httpx_client.get(
                    f'{agent_url}{AGENT_CARD_WELL_KNOWN_PATH}'
                )
                agent_card_data = self._agent_info_cache[agent_url] = (
                    agent_card_response.json()
                )

            # Create AgentCard from data
            agent_card = AgentCard(**agent_card_data)

            # Create A2A client with the agent card
            config = ClientConfig(
                httpx_client=httpx_client,
                supported_transports=[
                    TransportProtocol.jsonrpc,
                    TransportProtocol.http_json,
                ],
                use_client_preference=True,
            )

            factory = ClientFactory(config)
            client = factory.create(agent_card)

            # Create the message object
            message_obj = create_text_message_object(content=message)

            # Send the message and collect responses
            responses = []
            async for response in client.send_message(message_obj):
                responses.append(response)

            # The response is a tuple - get the first element (Task object)
            if (
                responses
                and isinstance(responses[0], tuple)
                and len(responses[0]) > 0
            ):
                task = responses[0][0]  # First element of the tuple

                # Extract text: task.artifacts[0].parts[0].root.text
                try:
                    return task.artifacts[0].parts[0].root.text
                except (AttributeError, IndexError):
                    return str(task)

            return 'No response received'


a2a_client = A2ASimpleClient()
print('A2A Simple Client created')

A2A Simple Client created


### Router Orchestrator

In [10]:
class RouterOrchestrator:
    """
    Router agent that orchestrates customer service by coordinating specialist agents.
    Uses an LLM to analyze queries, decide which agents to call, and synthesize responses.
    """

    def __init__(self, a2a_client: A2ASimpleClient):
        """
        Initialize the Router Orchestrator.

        Args:
            a2a_client: An A2ASimpleClient instance for agent communication.
        """
        self.client = a2a_client
        self.model = genai.GenerativeModel(LLM_MODEL)

        # URLs for the specialist agents
        self.DATA_AGENT_URL = CUSTOMER_DATA_AGENT_URL
        self.SUPPORT_AGENT_URL = SUPPORT_AGENT_URL

    async def call_agent(self, agent_name: str, query: str) -> str:
        """
        Execute an A2A call to a specialist agent.

        Args:
            agent_name: Name of the agent ("customer_data" or "support_agent").
            query: The query/message to send to the agent.

        Returns:
            str: The agent's response.
        """
        url = self.DATA_AGENT_URL if agent_name == "customer_data" else self.SUPPORT_AGENT_URL
        print(f"    >>> [A2A CALL] Connecting to {agent_name} at {url}...")

        try:
            # Use the existing a2a_client logic
            response = await self.client.create_task(url, query)
            return response
        except Exception as e:
            return f"Error calling agent: {str(e)}"

    async def process_query(self, user_query: str) -> str:
        """
        Process a customer query by coordinating specialist agents, by using the LLM to
        reason about the query, decide which agents to call, and form the final response.

        Args:
            user_query: The customer's query/request.

        Returns:
            str: The final response to the customer.
        """

        # Format the output by having === bars the length of the query
        header_text = f" USER QUERY: {user_query} "
        separator = "=" * len(header_text)

        print(f"\n{separator}")
        print(header_text)
        print(f"{separator}")

        # Set the maximum nunber of reasoning turns
        max_turns = 15

        # Define the Router's logic via detailed instructions
        system_prompt = """
        You are the Router Agent (Orchestrator) for a customer service system.
        You have two specialized sub-agents you can call via A2A tools:

        1. "customer_data"
        - Capabilities: Get customer details, list customers, update records, get ticket history, create tickets.

        2. "support_agent"
        - Capabilities: General support advice, troubleshooting, escalation decisions.

        Your Goal: Answer the user's request by coordinating these agents.

        CRITICAL INSTRUCTION FOR LISTS:
        - If the user asks for a list of customers with specific conditions (e.g., "active customers with open tickets"), do NOT check them one by one.
        - **BATCH YOUR REQUESTS:**
        1. Call customer_data to list active customers.
        2. Send a **SINGLE** message to customer_data requesting ticket history for **ALL** the retrieved customer IDs at once (e.g. "Get history for customer IDs 4, 5, 8, 10...").
        3. Filter the results yourself based on the returned data.

        RESPONSE FORMAT:
        You must strictly return a JSON object in this format (no markdown formatting):
        {
            "thought": "Explanation of your reasoning",
            "action": "call_agent" OR "final_answer",
            "agent_name": "customer_data" OR "support_agent" (only if action is call_agent),
            "content": "The specific query string to send to that agent" OR "The final text response to the user"
        }
        """

        messages = [
            {"role": "user", "parts": [system_prompt + f"\n\nUser Query: {user_query}"]}
        ]

        # Loop through reasoning turns, using try/except to handle any errors
        for i in range(max_turns):
            try:
                response = await self.model.generate_content_async(messages)
                response_text = response.text
            except Exception as e:
                print(f"[ROUTER ERROR]: Gemini generation failed: {e}")
                return "System Error"

            # Clean up the JSON response
            clean_text = response_text.replace("```json", "").replace("```", "").strip()

            try:
                plan = json.loads(clean_text)
            except json.JSONDecodeError:
                print(f"[ROUTER ERROR]: Could not parse JSON plan. Raw: {clean_text}")
                messages.append({"role": "user", "parts": ["Invalid JSON. Please return ONLY valid JSON."]})
                continue

            print(f"\n[ROUTER STEP {i+1}]: {plan['thought']}")

            if plan['action'] == "final_answer":
                print(f"\n[ROUTER]: Task Complete.")
                return plan['content']

            elif plan['action'] == "call_agent":
                # Execute A2A Call
                agent_response = await self.call_agent(plan['agent_name'], plan['content'])

                snippet = str(agent_response)[:200].replace('\n', ' ')
                print(f"   <<< [A2A RESPONSE]: {snippet}...")

                # Add context to history
                messages.append({"role": "model", "parts": [clean_text]})
                messages.append({"role": "user", "parts": [f"Result from {plan['agent_name']}: {agent_response}"]})

        return "Error: Maximum turns reached without final answer."

router = RouterOrchestrator(a2a_client)
print("Router created")

Router created


## **Server Setup**

In [11]:
def create_agent_a2a_server(agent, agent_card):
    """Create an A2A server for any ADK agent.

    Args:
        agent: The ADK agent instance
        agent_card: The ADK agent card

    Returns:
        A2AStarletteApplication instance
    """
    runner = Runner(
        app_name=agent.name,
        agent=agent,
        artifact_service=InMemoryArtifactService(),
        session_service=InMemorySessionService(),
        memory_service=InMemoryMemoryService(),
    )

    config = A2aAgentExecutorConfig()
    executor = A2aAgentExecutor(runner=runner, config=config)

    request_handler = DefaultRequestHandler(
        agent_executor=executor,
        task_store=InMemoryTaskStore(),
    )

    # Create A2A application
    return A2AStarletteApplication(
        agent_card=agent_card, http_handler=request_handler
    )

In [12]:
# Apply nest_asyncio
nest_asyncio.apply()

# Store server tasks
server_tasks: list[asyncio.Task] = []

async def run_agent_server(agent, agent_card, port) -> None:
    """Run a single agent server."""
    app = create_agent_a2a_server(agent, agent_card)

    config = uvicorn.Config(
        app.build(),
        host='127.0.0.1',
        port=port,
        log_level='warning',
        loop='none',
    )

    server = uvicorn.Server(config)
    await server.serve()

async def start_all_servers() -> None:
    """Start ONLY the Customer and Support servers."""
    tasks = [
        asyncio.create_task(
            run_agent_server(customer_data_agent, customer_data_agent_card, 10020)
        ),
        asyncio.create_task(
            run_agent_server(support_agent, support_agent_card, 10021)
        ),
        # REMOVED: router_agent server on 10040
    ]

    await asyncio.sleep(2)

    print('Agent servers started!')
    print('   - Customer Data Agent: http://0.0.0.0:10020')
    print('   - Support Agent:       http://0.0.0.0:10021')

    try:
        await asyncio.Event().wait()
    except KeyboardInterrupt:
        print('Shutting down servers...')

def run_servers_in_background() -> None:
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    loop.run_until_complete(start_all_servers())

server_thread = threading.Thread(target=run_servers_in_background, daemon=True)
server_thread.start()

time.sleep(3)

Agent servers started!
   - Customer Data Agent: http://0.0.0.0:10020
   - Support Agent:       http://0.0.0.0:10021


In [13]:
print('Customer Data Agent Card:')
print(customer_data_agent_card)
print('\nSupport Agent Card:')
print(support_agent_card)

Customer Data Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=False) default_input_modes=['text/plain'] default_output_modes=['application/json'] description='Fetches and updates customer data using MCP tools' documentation_url=None icon_url=None name='Customer Data Agent' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieve and update customer records', examples=['Get customer 42', 'Update customer 5 email', 'Show all customers'], id='customer_data_access', input_modes=None, name='Customer Data Access', output_modes=None, security=None, tags=['customer', 'data', 'database', 'lookup'])] supports_authenticated_extended_card=None url='http://127.0.0.1:10020' version='1.0'

Support Agent Card:
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifi

## **Testing and Evaluation**

### Customer Data Agent Testing

In [14]:
async def test_customer_data():
    response = await a2a_client.create_task(
        'http://127.0.0.1:10020',  # correct
        'Look up customer with id 1'
    )
    print(response)

asyncio.run(test_customer_data())

```json
[CUSTOMER_DATA_AGENT]: Calling get_customer for ID 1
```
```json
{"customer": {"id": 1, "name": "John Smith", "email": "john.smith@example.com", "phone": "555-123-4567", "status": "active"}}
```


### Support Agent Testing

In [15]:
async def test_support_agent():
    response = await a2a_client.create_task(
        'http://localhost:10021',  # correct
        'A customer says they cannot log in. What should they do?'
    )
    print("Support Agent Response:\n", response)

asyncio.run(test_support_agent())

Support Agent Response:
 [SUPPORT_AGENT]: Responding to user's login issue

If a customer is having trouble logging in, here are a few things they can try:

1.  **Double-check their email address and password:** Make sure there are no typos. Passwords are case-sensitive!
2.  **Reset their password:** If they've forgotten their password, they can use the "Forgot Password" link on the login page to reset it.
3.  **Check their internet connection:** A poor internet connection can sometimes cause login problems.
4.  **Clear their browser's cache and cookies:** Sometimes old cached data can interfere with the login process.
5.  **Try a different browser or device:** This can help determine if the problem is specific to their current setup.

If they've tried all of these steps and are still unable to log in, I'll need to request some account details to assist further.
[SUPPORT_AGENT]: Requesting customer data from router
```json
{
  "needs_customer_data": true,
  "reason": "Customer is havin

### Scenario 1: Simple Query (Single Agent)

Query: Get customer information for a specific ID

Expected Flow:
1. Router analyzes query
2. Router calls Customer Data Agent
3. Returns customer details

In [16]:
query_1 = "Get customer information for customer ID 5"

result_1 = await router.process_query(query_1)
print(f"\nFINAL RESPONSE:\n{result_1}")


 USER QUERY: Get customer information for customer ID 5 

[ROUTER STEP 1]: The user is requesting customer information for a specific customer ID. This task falls under the capabilities of the customer_data agent.
    >>> [A2A CALL] Connecting to customer_data at http://127.0.0.1:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling get_customer for ID 5 ```json {   "customer": {     "id": 5,     "name": "Charlie Brown",     "email": "charlie.brown@email.com",     "phone": "+1-555-0105",     "status...

[ROUTER STEP 2]: The customer_data agent returned the customer information for customer ID 5. Now I can provide the final answer to the user.

[ROUTER]: Task Complete.

FINAL RESPONSE:
Customer information for customer ID 5:
Name: Charlie Brown
Email: charlie.brown@email.com
Phone: +1-555-0105
Status: active
Created at: 2025-11-17 21:35:02
Updated at: 2025-11-17 21:35:02


### Scenario 2: Negotiation/Escalation (Multiple Agents)

Query: Customer wants to cancel subscription but has billing issues

Expected Flow:
1. Router detects multiple intents
2. Router calls Support Agent (for cancellation policy)
3. Router calls Customer Data Agent (for billing history)
4. Router synthesizes coordinated response

In [17]:
query_2 = "I want to cancel my subscription but I'm having billing issues. My customer ID is 1."

result_2 = await router.process_query(query_2)
print(f"\nFINAL RESPONSE:\n{result_2}")


 USER QUERY: I want to cancel my subscription but I'm having billing issues. My customer ID is 1. 

[ROUTER STEP 1]: The user wants to cancel their subscription and is having billing issues. This requires input from both customer_data (to cancel the subscription and check for billing issues) and potentially support_agent (to offer support or escalate if necessary). I'll start by gathering customer data and creating a ticket.
    >>> [A2A CALL] Connecting to customer_data at http://127.0.0.1:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling create_ticket for customer ID 1. ```json {   "customer_id": 1,   "issue": "Billing issues and subscription cancellation",   "priority": "medium" } ```...

[ROUTER STEP 2]: Now that a ticket has been created, I will forward the issue to a support agent.
    >>> [A2A CALL] Connecting to support_agent at http://127.0.0.1:10021...
   <<< [A2A RESPONSE]: [SUPPORT_AGENT]: This is a data retrieval request, not a support question. PASS_TO_CUSTO

### Scenario 3: Multi-Step Coordination (Complex Query)

Query: Show all active customers who have closed tickets

Expected Flow:
1. Router decomposes query into sub-tasks
2. Router calls Customer Data Agent to get active customers
3. Router calls Customer Data Agent again for ticket history
4. Router filters and synthesizes final report

In [23]:
query_3 = "Show me the names of all active customers who have closed tickets."

result_3 = await router.process_query(query_3)
print(f"\nFINAL RESPONSE:\n{result_3}")


 USER QUERY: Show me the names of all active customers who have closed tickets. 

[ROUTER STEP 1]: The user wants a list of active customers with closed tickets. I should first get a list of all active customers, then get the ticket history for all of those customers, and finally filter the list to only include customers with closed tickets.
    >>> [A2A CALL] Connecting to customer_data at http://127.0.0.1:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Calling list_customers with status active. ```json {"list_customers_response": {"result": {"content": [{"text": "{\n  \"success\": true,\n  \"count\": 24,\n  \"customers\": [\n ...

[ROUTER STEP 2]: Now that I have the list of active customers, I need to get their ticket history to filter the ones with closed tickets. I will send a request to customer_data with all the customer IDs at once.
    >>> [A2A CALL] Connecting to customer_data at http://127.0.0.1:10020...
   <<< [A2A RESPONSE]: [CUSTOMER_DATA_AGENT]: Getting customer 